Names.csv 
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę w której wyliczysz wzrost w stopach (feet)
* Odpowiedz na pytanie jakie jest najpopularniesze imię?
* Dodaj kolumnę i policz wiek aktorów 
* Usuń kolumny (bio, death_details)
* Zmień nazwy kolumn - dodaj kapitalizaję i usuń _
* Posortuj dataframe po imieniu rosnąco

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import DataFrame

filePath = "dbfs:/FileStore/tables/Files/names.csv"
namesDf = (spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(filePath))
print(namesDf.explain())
display(namesDf.limit(20))

== Physical Plan ==
FileScan csv [imdb_name_id#8342,name#8343,birth_name#8344,height#8345,bio#8346,birth_details#8347,date_of_birth#8348,place_of_birth#8349,death_details#8350,date_of_death#8351,place_of_death#8352,reason_of_death#8353,spouses_string#8354,spouses#8355,divorces#8356,spouses_with_children#8357,children#8358] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/Files/names.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<imdb_name_id:string,name:string,birth_name:string,height:int,bio:string,birth_details:stri...


None


imdb_name_id name birth_name height bio birth_details date_of_birth place_of_birth death_details date_of_death place_of_death reason_of_death spouses_string spouses divorces spouses_with_children children nm0000001 Fred Astaire Frederic Austerlitz Jr. 177 Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV. May 10, 1899 in Omaha, Nebraska, USA 1899-05-10 Omaha, Nebraska, USA June 22, 1987 in Los Angeles, California, USA  (pneumonia) 22.06.1987 Los Angeles, California, USA   pneumonia Robyn Smith (27 June 1980 - 22 June 1987) (his death)Phyllis Livingston Potter (12 July 1933 - 13 September 1954) (her death) (2 children) 2 0 1 2 nm0000002 Lauren Bacall Betty Joan Perske 174 Lauren Bacall was born Betty Joan Perske on September 16, 1924, in New York City. She was the daughter of Natalie Weinstein-Bacal, a Romanian Jewish immigrant, and William Perske, who was born in New Jersey, to Polish Jewish parents. Her family was middle-class, with her father working as a salesman and her mother as a secretary. They divorced when she was five and she rarely saw her father after that.As a school girl, she originally wanted to be a dancer, but later switched gears to head into acting. She studied at the American Academy of Dramatic Arts in New York, after attending She was educated at Highland Manor, a private boarding school in Tarrytown, New York (through the generosity of wealthy uncles), and then at Julia Richman High School, which enabled her to get her feet wet in some off-Broadway productions.Out of school, she entered modeling and, because of her beauty, appeared on the cover of Harper's Bazaar, one of the most popular magazines in the US. The wife of famed director Howard Hawks spotted the picture in the publication and arranged with her husband to have Lauren take a screen test. As a result, which was entirely positive, she was given the part of Marie Browning in Acque del Sud (1944), a thriller opposite Humphrey Bogart, when she was just 19 years old. This not only set the tone for a fabulous career but also one of Hollywood's greatest love stories (she married Bogart in 1945). It was also the first of several Bogie-Bacall films.After 1945's L'agente confidenziale (1945), Lauren received second billing in Il grande sonno (1946) with Bogart. The mystery, in the role of Vivian Sternwood Rutledge, was a resounding success. Although she was making one film a year, each production would be eagerly awaited by the public. In 1947, again with her husband, Lauren starred in the thriller La fuga (1947). The film kept movie patrons on the edge of their seats. The following year, she starred with Bogart, Edward G. Robinson, and Lionel Barrymore in L'isola di corallo (1948). The crime drama was even more of a nail biter than her previous film.In 1950, Lauren starred in Le foglie d'oro (1950), a drama set in 1894. It was a film of note because she appeared without her husband - her co-star was Gary Cooper. In 1953, Lauren appeared in her first comedy as Schatze Page in Come sposare un milionario (1953). The film, with co-stars Marilyn Monroe and Betty Grable, was a smash hit all across the 

In [ ]:
def cleanup_date(df: DataFrame, dcol: str) -> DataFrame:
       """Removes unnecessary text, prepares for the conversion, converts all the date forms to date"""
       df = (df.withColumn(dcol, F.regexp_replace(dcol, r"[^0-9.]", ""))
              .withColumn(dcol, F.regexp_replace(dcol, r"[-]", "."))
              .withColumn(dcol, 
                              F.when(F.col(dcol).rlike(r"^\d{2}\.\d{2}\.\d{4}$"), # DD.MM.YYYY 
                                     F.to_date(F.col(dcol), "dd.MM.yyyy"))
                               .when(F.col(dcol).rlike(r"^\d{4}\.\d{2}\.\d{2}$"), # YYYY.MM.DD
                                     F.to_date(F.col(dcol), "yyyy.MM.dd"))
                               .when(F.col(dcol).rlike(r"^\d{4}$"), # YYYY
                                     F.to_date(F.col(dcol), "yyyy"))
                            ))
       return df

namesDf1 = (namesDf.withColumn("current_epoch", F.current_timestamp())
                  .withColumn("height_feet", F.col("height") * 0.0328)
                  .drop("bio", "death_details"))

namesDf1 = cleanup_date(namesDf1, "date_of_birth")
namesDf1 = cleanup_date(namesDf1, "date_of_death")

namesDf1 = namesDf1.withColumn("age", F.floor(F.date_diff(F.coalesce(F.col("date_of_death"), F.current_date()), F.col("date_of_birth")) / 365))

new_names = [col.replace("_", " ").title().replace(" ", "") for col in namesDf1.columns]
namesDf1 = namesDf1.toDF(*new_names)

namesDf1 = namesDf1.sort(F.col("Name").asc())

# print(namesDf1.explain())
display(namesDf1.limit(20))

ImdbNameId,Name,BirthName,Height,BirthDetails,DateOfBirth,PlaceOfBirth,DateOfDeath,PlaceOfDeath,ReasonOfDeath,SpousesString,Spouses,Divorces,SpousesWithChildren,Children,CurrentEpoch,HeightFeet,Age
nm1001478,'Big' LeRoy Mobley,LeRoy King Mobley III,193,"April 1, 1973 in Atlantic City, New Jersey, USA",1973-04-01,"Atlantic City, New Jersey, USA",null,null,null,null,0,0,0,0,2025-03-29T09:48:24.928Z,6.330400000000001,52
nm0521811,'Ducky' Louie,Lawrence Louie,null,"July 22, 1931 in Berkeley, California, USA",1931-07-22,"Berkeley, California, USA",null,null,null,null,0,0,0,0,2025-03-29T09:48:24.928Z,null,93
nm0722372,'Little Billy' Rhodes,William H. Rhodes,null,"February 1, 1895 in Illinois, USA",null,"Illinois, USA",1967-07-24,"Hollywood, California, USA",stroke,null,0,0,0,0,2025-03-29T09:48:24.928Z,null,null
nm0946148,'Weird Al' Yankovic,Alfred Matthew Yankovic,183,"October 23, 1959 in Downey, California, USA",1959-10-23,"Downey, California, USA",null,null,null,Suzanne Krajewski (10 February 2001 - present) (1 child),1,0,1,1,2025-03-29T09:48:24.928Z,6.002400000000001,65
nm1265067,50 Cent,Curtis James Jackson III,183,"July 6, 1975 in Queens, New York City, New York, USA",1975-07-06,"Queens, New York City, New York, USA",null,null,null,null,0,0,0,0,2025-03-29T09:48:24.928Z,6.002400000000001,49
nm0553436,A Martinez,Adolph Larrue Martinez III,175,"September 27, 1948 in Glendale, California, USA",1948-09-27,"Glendale, California, USA",null,null,null,Leslie Bryans (17 July 1982 - present) (3 children)Mare Winningham (1981 - 29 January 1982) (divorced),2,1,1,3,2025-03-29T09:48:24.928Z,5.74,76
nm1100197,A. Baldwin Sloane,A. Baldwin Sloane,null,"August 28, 1872 in Baltimore, Maryland, USA",null,"Baltimore, Maryland, USA",1925-02-21,"Red Bank, New Jersey, USA",null,null,0,0,0,0,2025-03-29T09:48:24.928Z,null,null
nm0080406,A. Bhimsingh,A. Bhimsingh,null,"July 15, 1924 in Tirupati, Andhra Pradesh, India",1924-07-15,"Tirupati, Andhra Pradesh, India",1978-01-16,"Madras, Tamil Nadu, India",null,Sukumari (? - 16 January 1978) (his death) (1 child),1,0,1,1,2025-03-29T09:48:24.928Z,null,53
nm0770661,A. Hans Scheirl,Angela Hans Schierl,null,"1956 in Salzburg, Austria",1956-01-01,"Salzburg, Austria",null,null,null,null,0,0,0,0,2025-03-29T09:48:24.928Z,null,69
nm0072200,A. Jonathan Benny,A. Jonathan Benny,null,"November 4, 1970",1970-11-04,null,null,null,null,null,0,0,0,0,2025-03-29T09:48:24.928Z,null,54


## What is the most popular name?

1. John
2. Michael
3. David
4. Robert
5. Peter
---

In [ ]:
top_names = namesDf1.withColumn("FirstName", F.split(F.col("Name"), " ")[0]).groupby("FirstName").count().sort(F.col("count").desc())

top_names.show(5)

+---------+-----+
|FirstName|count|
+---------+-----+
|     John|  873|
|  Michael|  718|
|    David|  687|
|   Robert|  571|
|    Peter|  487|
+---------+-----+
only showing top 5 rows



Movies.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dodaj kolumnę która wylicza ile lat upłynęło od publikacji filmu
* Dodaj kolumnę która pokaże budżet filmu jako wartość numeryczną, (trzeba usunac znaki walut)
* Usuń wiersze z dataframe gdzie wartości są null

In [ ]:
filePath = "dbfs:/FileStore/tables/Files/movies.csv"
moviesDf = (spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(filePath))

moviesDf1 = (moviesDf.withColumn("current_epoch", F.current_timestamp())
                     .dropna()
                     .withColumn("budget", F.cast("int", F.regexp_replace(F.col("budget"), r"[^0-9]", "")))    
)

moviesDf1 = cleanup_date(moviesDf1, "date_published")
moviesDf1 = moviesDf1.withColumn("years_since_published", F.floor(F.date_diff(F.current_date(), F.col("date_published")) / 365))

display(moviesDf1.limit(20))

imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,current_epoch,years_since_published
tt0017136,Metropolis,Metropolis,1927,1928-10-01,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rogge, Fritz Rasp, Theodor Loos, Erwin Biswanger, Heinrich George, Brigitte Helm","In a futuristic city sharply divided between the working class and the city planners, the son of the city's mastermind falls in love with a working class prophet who predicts the coming of a savior to mediate their differences.",08.mar,156076,6000000,$ 1236166,$ 1349711,98.0,495.0,208.0,2025-03-29T09:48:46.32Z,96
tt0021749,Luci della città,City Lights,1931,1931-04-02,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, Al Ernest Garcia, Hank Mann, Charles Chaplin","With the aid of a wealthy erratic tippler, a dewy-eyed tramp who has fallen in love with a sightless flower girl accumulates money to be able to help her medically.",08.maj,162668,1500000,$ 19181,$ 46008,99.0,295.0,122.0,2025-03-29T09:48:46.32Z,94
tt0027977,Tempi moderni,Modern Times,1936,1937-03-12,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergman, Tiny Sandford, Chester Conklin, Hank Mann, Stanley Blystone, Al Ernest Garcia, Richard Alexander, Cecil Reynolds, Mira McKinney, Murdock MacQuarrie, Wilfred Lucas, Edward LeSaint, Fred Malatesta",The Tramp struggles to live in modern industrial society with the help of a young homeless woman.,08.maj,211250,1500000,$ 163577,$ 457688,96.0,307.0,115.0,2025-03-29T09:48:46.32Z,88
tt0029453,Il bandito della Casbah,Pépé le Moko,1937,1937-10-22,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,"Henri La Barthe, Henri La Barthe",Paris Film,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fernand Charpin, Lucas Gridoux, Gilbert Gil, Marcel Dalio, Charles Granval, Gaston Modot, René Bergeron, Paul Escoffier, Roger Legris, Jean Témerson, Robert Ozanne, Philippe Richard","A wanted gangster is both king and prisoner of the Casbah. He is protected from arrest by his friends, but is torn by his desire for freedom outside. A visiting Parisian beauty may just tempt his fate.",07.lip,6180,60000,$ 155895,$ 155895,98.0,46.0,55.0,2025-03-29T09:48:46.32Z,87
tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938-11-30,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti, Eddie Collins, Pinto Colvig, Marion Darlington, Billy Gilbert, Otis Harlan, Lucille La Verne, James MacDonald, Scotty Mattraw, Moroni Olsen, Purv Pullen, Harry Stockwell, Bill Thompson","Exiled into the dangerous forest by her wicked stepmother, a princess is rescued by seven dwarf miners who make her part of their household.",07.cze,177157,1499000,$ 184925486,$ 184925486,95.0,260.0,173.0,2025-03-29T09:48:46.32Z,86
tt0031381,Via col vento,Gone with the Wind,1939,1949-03-12,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh, Evelyn Keyes, Ann Rutherford, George Reeves, Fred Crane, Hattie McDaniel, Oscar Polk, Butterfly McQueen, Victor Jory, Everett Brown, Howard Hickman, Alicia Rhett, Leslie Howard",A manipulative woman and a roguish man conduct a turbulent romance during the American Civil War and Reconstruction periods.,08.sty,283975,3977000,$ 200852579,$ 402352579,97.0,881.0,197.0,2025-03-29T09:48:46.32Z,76
tt0031679,Mr. Smith va a Washington,

ratings.csv
* Dodaj kolumnę z wartością czasu wykonania notatnika w formacie epoch
* Dla każdego z poniższych wyliczeń nie bierz pod uwagę `nulls` 
* Kto daje lepsze oceny chłopcy czy dziewczyny dla całego setu
* Dla jednej z kolumn zmień typ danych do `long` 

In [ ]:
filePath = "dbfs:/FileStore/tables/Files/ratings.csv"
ratingsDf = (spark.read.format("csv")
              .option("header","true")
              .option("inferSchema","true")
              .load(filePath))

display(ratingsDf.limit(20))

imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,103,28,20,13,16,6.0,1.0,6.1,114.0,6.0,239.0,6.3,115.0,6.1,425.0,6.0,1.0,6.2,102.0,6.0,210.0,6.2,100.0,6.2,50.0,null,null,5.9,12.0,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,32,16,5,6,4,null,null,5.5,25.0,5.8,72.0,6.2,62.0,5.9,146.0,null,null,5.5,21.0,5.9,67.0,6.2,55.0,5.7,15.0,null,null,5.8,4.0,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,117,63,26,25,16,null,null,5.3,23.0,5.0,111.0,5.3,193.0,5.1,299.0,null,null,5.2,20.0,4.9,96.0,5.2,171.0,5.9,39.0,null,null,5.7,3.0,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0
tt0002199,5.7,484,5.8,6.0,33,15,48,80,123,77,36,20,18,34,null,null,5.6,35.0,5.6,177.0,5.8,168.0,5.7,354.0,null,null,5.6,32.0,5.7,159.0,5.8,151.0,5.8,31.0,null,null,6.3,3.0,5.1,13.0,6.5,15.0,5.5,67.0,5.9,173.0,5.6,181.0
tt0002423,6.8,753,6.8,7.0,80,65,105,209,142,80,27,13,4,28,3.0,1.0,6.4,59.0,6.7,287.0,7.0,276.0,6.6,530.0,3.0,1.0,6.4,41.0,6.6,240.0,6.8,238.0,7.4,93.0,null,null,6.5,16.0,7.2,40.0,8.2,34.0,6.2,88.0,6.6,139.0,6.8,455.0
tt0002445,6.2,273,6.2,6.0,15,8,30,74,66,40,22,6,2,10,6.0,1.0,5.9,34.0,6.2,99.0,6.3,69.0,6.2,191.0,6.0,1.0,5.8,29.0,6.2,85.0,6.3,64.0,6.3,21.0,null,null,6.5,5.0,6.3,11.0,6.2,5.0,5.7,41.0,5.8,40.0,6.2,154.0
tt0002452,6.7,198,7.1,7.0,53,7,23,35,30,24,12,5,3,6,null,null,7.4,28.0,6.7,104.0,6.5,39.0,6.7,153.0,null,null,7.3,24.0,6.6,92.0,6.5,33.0,7.0,19.0,null,null,8.3,3.0,6.8,10.0,6.8,6.0,5.4,25.0,6.7,25.0,6.8,135.0
tt0002461,5.5,225,5.4,6.0,9,6,22,34,53,40,23,8,4,26,null,null,5.9,11.0,5.3,95.0,5.8,72.0,5.5,168.0,null,null,5.9,11.0,5.2,88.0,5.8,63.0,7.7,15.0,null,null,null,null,7.2,6.0,9.7,9.0,5.4,37.0,5.8,109.0,5.4,68.0


In [ ]:
ratingsDf1 = (ratingsDf.withColumn("current_epoch", F.current_timestamp())
                       .dropna()
                       .withColumn("total_votes", F.col("total_votes").cast("long")))

display(ratingsDf1.limit(20))

imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,current_epoch
tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,28,9,1,5,14,7.2,4.0,6.0,38.0,5.7,50.0,6.6,35.0,6.2,97.0,7.0,1.0,5.9,24.0,5.6,36.0,6.7,31.0,6.0,35.0,7.3,3.0,5.9,14.0,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0,2025-03-29T09:49:12.64Z
tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,169,66,39,20,87,7.5,4.0,7.0,402.0,7.0,895.0,7.1,482.0,7.0,1607.0,8.0,2.0,7.0,346.0,7.0,804.0,7.0,396.0,7.2,215.0,7.0,2.0,7.0,52.0,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0,2025-03-29T09:49:12.64Z
tt0003740,7.1,3073,6.5,7.0,285,301,591,727,443,199,85,27,18,397,6.0,3.0,7.0,393.0,7.0,1126.0,7.2,1006.0,7.1,2149.0,6.0,2.0,7.0,323.0,7.0,976.0,7.3,799.0,6.9,402.0,6.0,1.0,6.8,67.0,6.9,134.0,6.8,194.0,6.9,177.0,7.0,1035.0,7.0,1332.0,2025-03-29T09:49:12.64Z
tt0004972,6.3,22213,6.4,7.0,3661,1741,3314,3963,2876,1928,978,701,577,2474,4.7,14.0,6.0,3183.0,6.3,8861.0,6.7,4901.0,6.4,14818.0,4.7,12.0,6.1,2623.0,6.4,7669.0,6.8,4104.0,5.7,2417.0,6.0,1.0,5.4,517.0,5.8,1106.0,5.9,725.0,6.5,355.0,6.3,7452.0,6.4,8306.0,2025-03-29T09:49:12.64Z
tt0005680,5.9,130,5.9,6.0,4,3,8,20,52,27,8,5,2,1,5.0,3.0,5.8,19.0,5.9,44.0,5.9,50.0,5.9,102.0,5.0,2.0,5.8,16.0,5.8,40.0,6.0,43.0,5.9,12.0,5.0,1.0,5.5,2.0,6.5,4.0,6.2,5.0,5.7,50.0,5.9,26.0,5.8,87.0,2025-03-29T09:49:12.64Z
tt0006206,7.3,4166,6.7,7.0,620,439,763,872,484,232,140,40,31,545,6.5,7.0,7.1,517.0,7.3,1661.0,7.3,1294.0,7.3,2835.0,6.8,6.0,7.1,414.0,7.3,1384.0,7.3,976.0,7.2,670.0,4.0,1.0,6.9,98.0,7.1,256.0,7.7,306.0,6.8,204.0,7.5,1350.0,7.2,1945.0,2025-03-29T09:49:12.64Z
tt0006864,7.8,13875,7.8,8.0,3477,2230,3214,2249,1179,605,340,181,133,267,7.9,8.0,7.8,1795.0,7.7,5451.0,7.8,3667.0,7.8,9441.0,8.0,7.0,7.9,1498.0,7.7,4734.0,7.8,2970.0,7.5,1632.0,7.0,1.0,7.2,276.0,7.4,660.0,8.0,662.0,7.5,321.0,7.7,4286.0,7.8,5954.0,2025-03-29T09:49:12.64Z
tt0009611,7.3,5895,6.7,7.0,552,564,1303,1742,724,228,69,35,22,656,5.0,4.0,7.3,833.0,7.3,2162.0,7.3,1730.0,7.3,4074.0,4.7,3.0,7.3,729.0,7.3,1888.0,7.3,1340.0,7.4,746.0,6.0,1.0,7.4,96.0,7.3,257.0,7.6,380.0,6.8,283.0,7.3,1734.0,7.3,2694.0,2025-03-29T09:49:12.64Z
tt0010323,8.1,55601,7.9,8.0,11426,11262,15971,8883,3517,1562,725,483,348,1424,8.5,47.0,8.1,10734.0,8.1,23386.0,8.0,8271.0,8.1,35834.0,8.5,38.0,8.1,8304.0,8.0,19321.0,8.0,6821.0,8.1,7660.0,8.8,7.0,8.1,2236.0,8.2,3807.0,8.1,1341.0,7.5,559.0,8.0,13136.0,8.1,25399.0,2025-03-29T09:49:12.64Z
tt0011130,7.0,4753,7.1,7.0,393,335,965,1666,816,326,107,54,35,56,7.0,4.0,6.9,546.0,7.0,1830.0,7.0,1428.0,7.0,3253.0,6.0,3.0,6.9,415.0,6.9,1541.0,7.0,1207.0,7.2,606.0,10.0,1.0,6.8,128.0,7.3,269.0,7.2,202.0,6.9,239.0,7.1,1733.0,6.9,1815.0,2025-03-29T09:49:12.64Z


In [ ]:
rates_avg = ratingsDf1.agg(F.avg("males_allages_avg_vote").alias("average_male_votes"), F.avg("females_allages_avg_vote").alias("average_female_votes"))

rates_avg.show()

+------------------+--------------------+
|average_male_votes|average_female_votes|
+------------------+--------------------+
| 6.175647515893578|   6.371356251471637|
+------------------+--------------------+

